In [ ]:
import anthropic
import json
import pandas as pd
import textwrap
import os
import re
from dotenv import load_dotenv

pd.set_option('display.float_format', lambda x: '%.3f' % x)
idx = pd.IndexSlice

load_dotenv()

client = anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

In [ ]:
ANIMALS = ['Cattle', 'Chickens', 'Ducks', 'Goats', 'Pigs',
           'Sheep', 'Carp', 'Other Fish', 'Salmon', 'Shrimp']

In [ ]:
df = pd.read_csv('data/Animal Welfare Grants - All.csv')
df["Amount"] = df["Amount"].replace("[$,]", "", regex=True).astype(float)
df.sort_values(["Year", "Amount"], ascending=False, inplace=True)
df.to_csv('data/grants.csv')

print(df.head().to_string())

In [ ]:
def classify(row):
    prompt = str(textwrap.dedent("""Recipient: "{{recipient}}"
Description: "{{description}}"
Step 1: Analyze the grant to determine the type of animal.

Look for keywords related to "Farmed" (e.g., livestock, poultry, aquaculture), "Wild" (e.g., wildlife, conservation), or "Companion" (e.g., pets, shelters) animals.
If no clear indication of animal type, mark as "Unknown".
Analysis: {animal_type_analysis}
Animal Type: {animal_type}

Step 2: Identify the specific animal mentioned in the grant, if any.

Search for mentions of "Cattle", "Chickens", "Ducks", "Goats", "Pigs", "Sheep", "Carp, "Salmon", or "Shrimp".
If the grant benefits fish but does not specify a type, mark as "Other Fish".
If no specific animal is mentioned or if the animal doesn't match the listed categories, mark as "Not Specified".
Analysis: {specific_animal_analysis}
Specific Animal: {specific_animal}

Step 3: Determine the specific country mentioned in the grant, if any.

Look for mentions of country names or regions in the recipient or description.
If no specific country is mentioned, mark as "Not Specified".
Avoid false positives by ensuring the country name is in the proper context.
Analysis: {country_analysis}
Country: {country}

Final Classification:
                                 
{
    "animal_type": "{animal_type}",
    "specific_animal": "{specific_animal}",
    "country": "{country}"
}
    """)).replace('{{recipient}}', str(row['Recipient'])).replace('{{description}}', str(row['Description']))
    # print(prompt)
    try:
        response = client.messages.create(
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1024,
            temperature=0.0,
            model="claude-3-haiku-20240307")
        result = response.content[0].text.strip()
        json_match = re.search(r'({.*?})\s*$', result, re.DOTALL)
        if json_match:
            classification_json = json_match.group(1)
            print("Final Classification:", classification_json)
            return json.loads(classification_json)
        else:
            print("No JSON object found in the response.")
        return {}
        # return pd.Series(json.loads(result))
    except Exception as e:
        print(e)
        return {}
        # return pd.Series({'Animal': 'Unknown', 'Country': 'Unknown'})

In [ ]:
row = df.iloc[15]
# print(row)
result = classify(row)

In [ ]:
result

In [ ]:
df_test = df.iloc[:10]

In [ ]:
c = df.apply(lambda x: classify(x), axis=1)

In [ ]:
df[c.columns] = c

In [ ]:
df

In [ ]:
pt = df.pivot_table(index='Country', columns='Animal', values='Amount', aggfunc='sum', fill_value=0)
print(pt.to_string())

In [ ]:
s = pt.sum()
print(s)

In [ ]:
(s['Carp'] + s['Other Fish'] + s['Salmon'])/(-1951699833.697 + -1897440352.025 +  -35457490.468)

In [ ]:
s['Chickens']/-4208769312.000

In [ ]:
s['Shrimp']/-2018785541.017

In [ ]:
s['Pigs']/-202723782.386

In [ ]:
s['Cattle']/-156494452.036